In [1]:
import sys
import os
base_dir = "/home/jingpei/Desktop/CtRNet-robot-pose-estimation"
sys.path.append(base_dir)

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

from utils import *
from models.CtRNet import CtRNet
from models.BPnP import batch_transform_3d



In [2]:
import argparse
from imageloaders.DREAM import ImageDataLoaderReal, load_camera_parameters
parser = argparse.ArgumentParser()

args = parser.parse_args("")

args.data_folder = "/media/jingpei/DATA/DREAM/data/real/panda-3cam_realsense"
args.base_dir = "/home/jingpei/Desktop/CtRNet-robot-pose-estimation"
args.use_gpu = True
args.trained_on_multi_gpus = True
args.keypoint_seg_model_path = os.path.join(args.base_dir,"weights/panda/panda-3cam_realsense/net.pth")
args.urdf_file = os.path.join(args.base_dir,"urdfs/Panda/panda.urdf")

args.robot_name = 'Panda' # "Panda" or "Baxter_left_arm"
args.n_kp = 7
args.scale = 0.5
args.height = 480
args.width = 640
args.fx, args.fy, args.px, args.py = load_camera_parameters(args.data_folder)

# scale the camera parameters
args.width = int(args.width * args.scale)
args.height = int(args.height * args.scale)
args.fx = args.fx * args.scale
args.fy = args.fy * args.scale
args.px = args.px * args.scale
args.py = args.py * args.scale

In [3]:
CtRNet = CtRNet(args)

Loading keypoint segmentation model from /home/jingpei/Desktop/CtRNet-robot-pose-estimation/weights/panda/panda-3cam_realsense/net.pth
Camera intrinsics: [[307.76196289   0.         164.13032532]
 [  0.         307.60958862 125.89585114]
 [  0.           0.           1.        ]]
Robot model: Panda


In [4]:
trans_to_tensor = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = ImageDataLoaderReal(data_folder = args.data_folder, scale = args.scale, trans_to_tensor = trans_to_tensor)

In [5]:
from tqdm.notebook import tqdm

err_3d_list = list()
err_2d_list = list()

for data_n in tqdm(range(len(dataset))):
    img, joint_angles, keypoints = dataset.get_data_with_keypoints(data_n) 

    if args.use_gpu:
    
        img = img.cuda()

    cTr, points_2d, segmentation = CtRNet.inference_single_image(img, joint_angles.cpu().squeeze())

    # get ground truth
    points_2d_gt = list()
    points_3d_gt = list()
    for i in range(len(keypoints)):
        points_2d_gt.append(keypoints[i]['projected_location'])
        points_3d_gt.append(keypoints[i]['location'])

    points_2d_gt = np.array(points_2d_gt)
    points_3d_gt = np.array(points_3d_gt)


    # compute 3d keypoints
    _,t_list = CtRNet.robot.get_joint_RT(joint_angles.cpu().squeeze())
    points_3d = torch.from_numpy(t_list).float().cuda()

    points_3d_pred = to_np(batch_transform_3d(cTr, points_3d)).squeeze()
    points_3d_pred = points_3d_pred[[0,2,3,4,6,7,8]]


    if points_3d_pred[0,-1] < 0:
        points_3d_pred = -points_3d_pred

    err_3d = np.linalg.norm(points_3d_pred - points_3d_gt,axis=1)
    err_3d = np.mean(err_3d)
    err_3d_list.append(err_3d)


  0%|          | 0/5944 [00:00<?, ?it/s]

In [6]:
err_3d_list = np.array(err_3d_list).flatten()
err_3d_list.sort()

In [7]:
ADD = list()
for i in range(1000):
    num = np.sum(err_3d_list < i/10000.0) / err_3d_list.shape[0]
    ADD.append(num)

np.sum(ADD)/len(ADD)

0.9079485195154777

In [8]:
np.mean(err_3d_list)

0.0095501947591769